### A little inspection and cleanup of taxi data

Jeff Hale

In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz")
df

/tmp/ipykernel_97458/3321978667.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz")


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.00,3.00,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.00,0.50,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.00,0.50,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.00,0.50,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.50,0.50,0.5,4.06,0.0,0.3,24.36,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369760,NaN,2021-01-25 08:32:04,2021-01-25 08:49:32,NaN,8.80,NaN,NaN,135,82,NaN,21.84,2.75,0.5,0.00,0.0,0.3,25.39,0.0
1369761,NaN,2021-01-25 08:34:00,2021-01-25 09:04:00,NaN,5.86,NaN,NaN,42,161,NaN,26.67,2.75,0.5,0.00,0.0,0.3,30.22,0.0
1369762,NaN,2021-01-25 08:37:00,2021-01-25 08:53:00,NaN,4.45,NaN,NaN,14,106,NaN,25.29,2.75,0.5,0.00,0.0,0.3,28.84,0.0
1369763,NaN,2021-01-25 08:28:00,2021-01-25 08:50:00,NaN,10.04,NaN,NaN,175,216,NaN,28.24,2.75,0.5,0.00,0.0,0.3,31.79,0.0


In [6]:
df.to_csv("yellow_tripdata_2021-01_noindex.csv.gz", compression="gzip", index=False)

In [4]:
df.to_csv("yellow_tripdata_2021-01_index.csv.gz", compression="gzip", index=True)


In [5]:
df.to_parquet("yellow_tripdata_2021-01_noindex.parquet", compression="gzip", index=False)


In [10]:
import yaml
ff = '/home/michal/Projects/data-engineering-zoomcamp/cohorts/2023/week_2_workflow_orchestration/code/schemas/data_schemas.yaml'
color = 'fhv'

with open(ff, 'rb') as f:
    color_schema = yaml.safe_load(f)[color]

print(color_schema)

color_schema_lower = {k.lower(): v for k, v in color_schema.items()}
color_schema_lower



{'dispatching_base_num': 'string', 'pickup_datetime': 'datetime64[ns]', 'dropOff_datetime': 'datetime64[ns]', 'PUlocationID': 'Int64', 'DOlocationID': 'Int64', 'SR_Flag': 'Int64', 'Affiliated_base_number': 'str'}


{'dispatching_base_num': 'string',
 'pickup_datetime': 'datetime64[ns]',
 'dropoff_datetime': 'datetime64[ns]',
 'pulocationid': 'Int64',
 'dolocationid': 'Int64',
 'sr_flag': 'Int64',
 'affiliated_base_number': 'str'}

We should fix the mixed dtypes

In [50]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

## Transforms
### Change the date columns from string to datetime format

In [51]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [52]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-06-01 00:13:26,2021-06-01 00:17:14,1.0,0.9,1.0,N,186,50,1.0,5.0,3.00,0.5,2.2,0.00,0.3,11.0,2.5
1,1.0,2021-06-01 00:32:23,2021-06-01 01:07:04,1.0,23.0,1.0,N,132,18,2.0,61.5,1.75,0.5,0.0,6.55,0.3,70.6,0.0


Because there are missing values, pandas converts to a float. We could have specified we wanted nullable integer dtype, but float is fine.

### Write out cleaned file

In [14]:
df.to_parquet("2022-07_yellow_cleaned.parquet", compression='gzip')

The end

In [1]:
from hashlib import sha1

def compute_hash(email):
    return sha1(email.lower().encode('utf-8')).hexdigest()

compute_hash("MGasiorowski82@gmail.com")

'10eee6db0d16be4e024d96b8f33760398bc38677'